In [ ]:
import numpy as np
import pandas as pd 
import itertools
import xlrd # !pip install xlrd==1.2.0
import functools
import re
import random
import os
import requests
import shutil

from collections import Counter, defaultdict

In [ ]:
os.chdir('/content/drive/MyDrive/Free time (Aaron Presser): Poverty prediction project (02 14 2022)/')

In [ ]:
def csv_reader(data_path):
  data_csv = pd.read_csv(data_path)
  data_df = pd.DataFrame(data_csv)
  return data_df

In [ ]:
def get_df_name(df):
  name =[x for x in globals() if globals()[x] is df][0]
  return name

def write_csv(df):
  os.makedirs('Output_csv_files', exist_ok=True)
  name = get_df_name(df)
  df.to_csv(f"Output_csv_files/{name}.csv")
  print("Created file: \"{}.csv\"".format(name))
  return print("Succsess!")

In [ ]:
pd.read_csv('https://query.data.world/s/e3inb7voipg5l7be5csremp2py3xwv')

,ZIP,STCOUNTYFP,CITY,STATE,COUNTYNAME,CLASSFP
0,36091,1001,Verbena,AL,Autauga County,H1
1,36758,1001,Plantersville,AL,Autauga County,H1
2,36006,1001,Billingsley,AL,Autauga County,H1
3,36067,1001,Prattville,AL,Autauga County,H1
4,36701,1001,Selma,AL,Autauga County,H1
...,...,...,...,...,...,...
53957,820,78010,Christiansted,VI,St. Croix Island,H4
53958,840,78010,Frederiksted,VI,St. Croix Island,H4
53959,850,78010,Kingshill,VI,St. Croix Island,H4
53960,830,78020,St john,VI,St. John Island,H4


In [ ]:
walmart_path = 'Outside_data/Walmart_United_States_&_Canada.csv'
walmart_csv = pd.read_csv(walmart_path, encoding='latin-1')
walmart_df_raw = pd.DataFrame(walmart_csv)
walmart_df_raw

,-114.005671,51.262567,"Walmart SC; #1050,","Gas,2881 Main St SW, Airdrie, AB,T4B 3G5"
0,-111.900542,50.577939,"Walmart; #3658,","917 3rd St W, Brooks, AB,T1R 1L5"
1,-114.062149,50.900758,"Walmart SC; #3151,","Gas,100 - 310 Shawville Blvd, SE, Calgary, AB,..."
2,-114.039133,51.107253,"Walmart SC; #3013,","1110 57th Ave NE, Calgary, AB,T2E 9B7"
3,-114.138488,51.040871,"Walmart; #3009,","Gas,1212 37 St SW, Calgary, AB,T3C 1S3"
4,-113.911590,51.040090,"Walmart SC; #1136,","255 E Hills Blvd, Calgary, AB,T2A 4X7"
...,...,...,...,...
7148,-107.209281,41.792084,"Walmart SC; #4471,","Gas,2390 E Cedar St; I-80 Exit 214, Rawlins, W..."
7149,-108.379227,43.042858,"Walmart SC; #1457,","1733 N Federal Blvd, Riverton, WY,82501"
7150,-109.251020,41.579761,"Walmart SC; #1461,","201 Gateway Blvd; I-80 Exit 102, Rock Springs,..."
7151,-106.940967,44.779474,"Walmart SC; #1508,","1695 Coffeen Ave; I-90 Exit 25, Sheridan, WY,8..."


In [ ]:
# real_colnames = ['Latitude', 'Longitude', 'Location', 'Address']
real_colarrays = []
for col in walmart_df_raw.columns:
  col_asarray = walmart_df_raw[col].to_numpy()
  real_colarrays.append(col_asarray)

In [ ]:
data_dict = {'Latitude': real_colarrays[0], 'Longitude': real_colarrays[1], 'Location': real_colarrays[2], 'Address': real_colarrays[3]}
walmart_df = pd.DataFrame(data_dict)
walmart_df.sample(n=20, axis=0)

,Latitude,Longitude,Location,Address
4185,-55.651622,48.947507,"Walmart; #3036,","19 Cromer Ave, Grand Falls-Windsor, NL,A2A 1X3"
6348,-96.683620,33.159065,"Murphy USA; #7039,","Gas/Diesel,5251 McKinney Ranch Pkwy, McKinney,..."
4926,-79.484682,44.066859,"Walmart SC; #3062,","17940 Yonge St, Newmarket, ON,L3Y 8S4"
7,-113.966699,50.930235,"Walmart SC; #3650,","4705 130th Ave SE, Calgary, AB,T2Z 4J2"
1727,-84.203576,30.886492,"Walmart SC; #0858,","Gas/Diesel,361 8th Ave NE, Cairo, GA,39828"
3631,-89.031514,34.493501,"Walmart SC; #0153,","Gas/Diesel,202 Park Plaza Dr; I-22 Exit 61, Ne..."
4079,-103.677246,41.879671,"Walmart SC; #0867,","Gas,3322 Ave I, Scottsbluff, NE,69361"
4568,-84.317487,39.311474,"Walmart SC; #1441,","5303 Bowen Dr; I-71 Exit 19, Mason, OH,45040"
5098,-77.203651,39.843928,"Walmart; #1537,","1270 York Rd, Gettysburg, PA,17325"
1750,-84.947796,32.541855,"Sam's Club; #8213,","Gas/Diesel,5448 A Whittlesey Blvd, Columbus, G..."


In [ ]:
walmart_df = pd.DataFrame(data_dict)

def remove_state_abbrevs(df, foreign_states=None):
  if foreign_states == None:
    foreign_states = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'ON', 'PE', 'QC', 'SK', 'YT', 'MEX']
  foreign_states_precise = [", " + i + "," for i in foreign_states]

  df = df[~df.Address.str.contains('|'.join(foreign_states_precise), regex=True)]

  df.reset_index(inplace=True, drop=True)
  return df

walmart_df = remove_state_abbrevs(walmart_df)

In [ ]:
df_cleaning.shape

(6753, 4)

In [ ]:
address_lengths = []
addresses = walmart_df['Address']
for address in addresses:
  iter = address.split(',')
  address_lengths.append(len(iter))

print("The maximum length of the strings in the Address column is {}".format(np.array(address_lengths).max()))
print("The minimum length of the strings in the Address column is {}".format(np.array(address_lengths).min()))

The maximum length of the strings in the Address column is 6
The minimum length of the strings in the Address column is 3


In [ ]:
for address in addresses:
  iter = address.split(',')
  if (len(iter) == 3): print(iter)

['3850 Sheppard Ave E', ' Scarborough ON', 'M1T 3L4']


In [ ]:
for address in addresses:
  iter = address.split(',')
  if (len(iter) == 3): walmart_df.drop([True for iter in walmart_df if (len(iter) == 3)])

row_lengths = []
for address in addresses:
  iter = address.split(',')
  row_lengths.append(len(iter))

print("The maximum length of the strings in the Address column is {}".format(np.array(row_lengths).max()))
print("The minimum length of the strings in the Address column is {}".format(np.array(row_lengths).min()))

The maximum length of the strings in the Address column is 6
The minimum length of the strings in the Address column is 3


In [ ]:
addresses = walmart_df['Address']

first_segments = [] ; zipcode_segments = []
for address in addresses:
  iter = address.split(',')
  first_segments.append(iter[0])
  zipcode_segments.append(iter[-1])

car_juice = ['1' if re.match('Gas', x) else '0' for x in first_segments]
zip_codes = [x if re.match(r'(\d+){5}', x) else '' for x in zipcode_segments]

walmart_df['Car Juice'] = car_juice
walmart_df['Zip Code'] = zip_codes

walmart_df.head(20)

,Latitude,Longitude,Location,Address,Car Juice,Zip Code
0,-149.881507,61.192165,"Walmart SC; #2070,","3101 A St, Anchorage, AK,99503",0,99503
1,-149.742550,61.212740,"Walmart SC; #4359,","7405 Debarr Rd, Anchorage, AK,99504",0,99504
2,-149.868161,61.140234,"Walmart SC; #2071,","8900 Old Seward Hwy, Anchorage, AK,99515",0,99515
3,-149.535324,61.309780,"Walmart SC; #2188,","18600 Eagle River Rd, Eagle River, AK,99577",0,99577
4,-147.688971,64.857356,"Walmart SC; #2722,","537 Johansen Expy, Fairbanks, AK,99701",0,99701
5,-151.226631,60.564167,"Walmart SC; #4474,","10096 Kenai Spur, Kenai, AK,99611",0,99611
6,-131.720364,55.376113,"Walmart; #2710,","4230 Don King Rd, Ketchikan, AK,99901",0,99901
7,-152.364450,57.811389,"Walmart; #2711,","2911 Mill Bay Rd, Kodiak, AK,99615",0,99615
8,-149.362998,61.568711,"Walmart SC; #2074,","1350 S Seward Meridian Pkwy, Wasilla, AK,99654",0,99654
9,-86.924324,33.580015,"Walmart SC; #1158,","2473 Hackworth Rd, Adamsville, AL,35214",0,35214


In [ ]:
write_csv(walmart_df)

Created file: "walmart_df.csv"
Suck-sess!


In [ ]:
### End of Walmart dataframe cleaning

In [ ]:
fastfood_df_raw = csv_reader('Outside_data/Datafiniti_Fast_Food_Restaurants_May19.csv')

In [ ]:
# Delete
fastfood_dict = {k: v for k, v in sorted(Counter(fastfood_df_raw['name']).items(), key=lambda item: item[1], reverse=True)}
fastfood_dict

{"McDonald's": 765,
 'Subway': 667,
 'SUBWAY': 543,
 'Taco Bell': 529,
 'Pizza Hut': 474,
 'Burger King': 446,
 'KFC': 376,
 'Dairy Queen': 251,
 "Wendy's": 245,
 "Domino's Pizza": 227,
 'Sonic Drive-In': 223,
 "Arby's": 219,
 'McDonalds': 151,
 'Panera Bread': 140,
 'Chipotle Mexican Grill': 139,
 "Jimmy John's": 123,
 'Jack in the Box': 120,
 "Hardee's": 116,
 'Panda Express': 107,
 'Waffle House': 103,
 'Chick-fil-A': 77,
 'Chick-Fil-A': 76,
 "Moe's Southwest Grill": 73,
 "Jersey Mike's Subs": 69,
 'Wendys': 67,
 'Kfc': 57,
 'Popeyes Louisiana Kitchen': 52,
 "Steak 'n Shake": 51,
 "Church's Chicken": 51,
 'Quiznos': 48,
 'SONIC Drive-In': 48,
 'SONIC Drive In': 45,
 'Dominos Pizza': 43,
 'Five Guys Burgers Fries': 41,
 'Dairy Queen Grill Chill': 40,
 'Checkers': 40,
 'Taco Time': 39,
 'Krispy Krunchy Chicken': 39,
 'Whataburger': 38,
 'Firehouse Subs': 38,
 'QDOBA Mexican Eats': 36,
 "Long John Silver's": 35,
 "Culver's": 35,
 'Bob Evans': 35,
 "Bojangles' Famous Chicken 'n Biscuits

In [ ]:
# Delete
def they_match(k: str, nk: str) -> bool:
  '''Checking if two strings match when simplified in some ways'''
  k_low = k.lower()
  nk_low = nk.lower()
  regex = re.compile('[a-z]+| - ')
  k_low_list = ''.join(item for item in regex.findall(k_low))
  nk_low_list = ''.join(item for item in regex.findall(nk_low))
  return k_low_list == nk_low_list

old_dict = fastfood_dict
new_dict = {}

for old_key in old_dict:
    for new_key in new_dict:
        if they_match(old_key, new_key):
            if old_dict[old_key] > max(new_dict[new_key]):
                new_dict[old_key] = new_dict.pop(new_key)
                new_dict[old_key].append(old_dict[old_key])
            else:
                new_dict[new_key].append(old_dict[old_key])
            break
    else:
        new_dict[old_key] = [old_dict[old_key]]

for (k,v) in new_dict.items():
  print('{}: {}'.format(k, sum(v)))

print(len(new_dict))
sum(Counter(fastfood_df_raw['name']).values())

McDonald's: 921
Subway: 1210
Taco Bell: 529
Pizza Hut: 474
Burger King: 448
KFC: 433
Dairy Queen: 252
Wendy's: 312
Domino's Pizza: 270
Sonic Drive-In: 323
Arby's: 249
Panera Bread: 140
Chipotle Mexican Grill: 139
Jimmy John's: 153
Jack in the Box: 120
Hardee's: 138
Panda Express: 107
Waffle House: 103
Chick-fil-A: 174
Moe's Southwest Grill: 77
Jersey Mike's Subs: 72
Popeyes Louisiana Kitchen: 78
Steak 'n Shake: 59
Church's Chicken: 63
Quiznos: 48
Five Guys Burgers Fries: 49
Dairy Queen Grill Chill: 65
Checkers: 40
Taco Time: 42
Krispy Krunchy Chicken: 39
Whataburger: 38
Firehouse Subs: 38
QDOBA Mexican Eats: 38
Long John Silver's: 39
Culver's: 40
Bob Evans: 35
Bojangles' Famous Chicken 'n Biscuits: 43
Del Taco: 30
Jack's: 30
Carl's Jr.: 55
White Castle: 28
Krystal: 27
Fox's Pizza Den: 30
Jack's Family Restaurants: 25
Orange Julius: 24
Dunkin Donuts: 43
Sbarro: 23
Taco John's: 36
Five Guys: 21
AW All-American Food: 20
Charley's Grilled Subs: 20
Boston Market: 19
McAlister's Deli: 31
Rou

10000

In [ ]:
# (?)Delete
names = fastfood_df_raw['name'].tolist()

for i in range(len(names)):
  for key in new_dict:
    if they_match(names[i], key):
      del names[i]
      names.insert(i, key)
      break

fastfood_df = fastfood_df_raw.assign(name=names)
fastfood_df

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.396290,-104.696990,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.081350,-108.586890,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.091480,-108.554110,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.763690,-104.776710,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.740440,-104.926360,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,2016-05-31T13:33:45Z,2018-12-03T09:04:19Z,1803 Carolina Ave,"American Restaurants,Caterers,Soul Food Restau...",Accommodation & Food Services,Bessemer,US,us/al/bessemer/1803carolinaave/785498836,33.399574,-86.951050,Church's Chicken Of Birmingham,35020,AL,http://www.yellowpages.com/bessemer-al/mip/chu...,http://www.churchs.com
9996,AV1XW1fF3D1zeR_xFKHL,2017-07-18T20:19:50Z,2018-12-02T00:08:36Z,8511 Cantilever Way Ste 105,"Mexican Restaurants,Caterers,Delicatessens,Iri...",Accommodation & Food Services,Raleigh,US,us/nc/raleigh/8511cantileverwayste105/-1596320866,35.884500,-78.737541,Luxury Box,27613-6996,NC,https://www.bbb.org/us/nc/raleigh/profile/spor...,"http://www.theluxuryboxofraleigh.com,http://th..."
9997,AV5UVbkiRxPSIh2RiQxV,2017-09-05T23:05:50Z,2018-12-01T07:38:03Z,2601 Plover Rd,"Carry-out food,Hot dog restaurants,Fast Food R...",Accommodation & Food Services,Plover,US,us/wi/plover/2601ploverrd/-1161002137,44.453180,-89.543170,McDonald's,54467,WI,https://www.yellowpages.com/plover-wi/mip/mcdo...,"http://mcdonalds.com,http://www.mcdonalds.com/..."
9998,AVwc9BxYByjofQCxj_GW,2016-03-31T02:43:12Z,2018-12-01T07:35:22Z,170 E Main St,"Carry-out food,Caterers,Restaurant delivery se...",Accommodation & Food Services,Hendersonville,US,us/tn/hendersonville/170emainst/-66712705,36.308262,-86.607339,Pizza Hut,37075,TN,https://foursquare.com/v/pizza-hut/4c5fffd43a3...,http://www.pizzahut.com


In [ ]:
# Delete
counted = Counter(",".join(fastfood_df['categories']).split(","))
{k: v for k, v in sorted(counted.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
def process_contains(df, column, contains, replace):
  '''Takes a df as input, creates binary columns out of shared attributes
  
  Args:
    df - input dataframe
    column - Specifies the column you want to examine entries of
    contains - If the column entries contain these things, apply the function
    replace - What to replace the 'contains' argument with
    '''
  if contains is None:
    contains = ['Cater', 'Delic', 'Hot dog', 'Mexican', 'Sandwich', 'Coffee', 'Veg', 'Pizza']
  else: contains = set(df[f"{contains}"])
  
  if replace is None:
    replace = ['Catering', 'Deli', 'Hot dogs', 'Mexican', 'Sandwich', 'Coffee', 'Vegetarian', 'Pizza']
  else: replace = set(df[f"{replace}"])
  
  if column is None:
    column = 'categories'

  variable_dict = {}
  for i, j, k in zip(contains, replace, range(len(contains))):
    boolean_mask = df[f"{column}"].str.contains(i)
    variable_dict[f'{j}'] = boolean_mask
  
  df.drop(columns=[f"{column}"], inplace=True)
  variable_df = pd.DataFrame(variable_dict)
  variable_df.replace({False: 0, True: 1}, inplace=True)

  return df.join(variable_df)

In [ ]:
# Delete
fastfood_df['Car Juice']

KeyError: ignored

In [ ]:
fastfood_df = process_contains(fastfood_df, column = 'Location', contains=set(fastfood_df['Location']), replace=set(fastfood_df['Location']))
fastfood_df

KeyError: ignored

In [ ]:
fastfood_df = process_contains(fastfood_df)
fastfood_df

,id,dateAdded,dateUpdated,address,primaryCategories,city,country,keys,latitude,longitude,...,sourceURLs,websites,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,Pizza
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.396290,-104.696990,...,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...,0,0,0,0,0,0,0,0
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.081350,-108.586890,...,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com,0,0,0,1,0,0,0,0
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.091480,-108.554110,...,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com,0,0,0,0,1,0,0,0
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.763690,-104.776710,...,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com,0,0,0,1,0,0,0,0
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.740440,-104.926360,...,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,2016-05-31T13:33:45Z,2018-12-03T09:04:19Z,1803 Carolina Ave,Accommodation & Food Services,Bessemer,US,us/al/bessemer/1803carolinaave/785498836,33.399574,-86.951050,...,http://www.yellowpages.com/bessemer-al/mip/chu...,http://www.churchs.com,1,0,0,0,0,0,0,0
9996,AV1XW1fF3D1zeR_xFKHL,2017-07-18T20:19:50Z,2018-12-02T00:08:36Z,8511 Cantilever Way Ste 105,Accommodation & Food Services,Raleigh,US,us/nc/raleigh/8511cantileverwayste105/-1596320866,35.884500,-78.737541,...,https://www.bbb.org/us/nc/raleigh/profile/spor...,"http://www.theluxuryboxofraleigh.com,http://th...",1,1,0,1,1,0,0,0
9997,AV5UVbkiRxPSIh2RiQxV,2017-09-05T23:05:50Z,2018-12-01T07:38:03Z,2601 Plover Rd,Accommodation & Food Services,Plover,US,us/wi/plover/2601ploverrd/-1161002137,44.453180,-89.543170,...,https://www.yellowpages.com/plover-wi/mip/mcdo...,"http://mcdonalds.com,http://www.mcdonalds.com/...",0,0,1,0,0,0,0,0
9998,AVwc9BxYByjofQCxj_GW,2016-03-31T02:43:12Z,2018-12-01T07:35:22Z,170 E Main St,Accommodation & Food Services,Hendersonville,US,us/tn/hendersonville/170emainst/-66712705,36.308262,-86.607339,...,https://foursquare.com/v/pizza-hut/4c5fffd43a3...,http://www.pizzahut.com,1,0,0,0,0,0,0,1


In [ ]:
col_dict = {'dateAdded': 'WhenAdded', 'dateUpdated': 'LastUpdated',\
                        'address': 'Address', 'primaryCategories': 'PrimaryCategories',\
                        'city': 'City','latitude': 'Latitude', 'longitude': 'Longitude',\
                        'name': 'Restaurant Name', 'postalCode': 'Zip Code',\
                        'province': 'State', 'Catering': 'Catering', 'Deli': 'Deli',\
                        'Hot dogs': 'Hot dogs', 'Mexican': 'Mexican', 'Sandwich': 'Sandwich',\
                        'Coffee': 'Coffee', 'Vegetarian': 'Vegetarian', 'Pizza': 'Pizza'}

fastfood_df.rename(columns=col_dict, inplace=True)

cols = list(col_dict.values())
fastfood_df = fastfood_df[cols]
fastfood_df

,WhenAdded,LastUpdated,Address,PrimaryCategories,City,Latitude,Longitude,Restaurant Name,Zip Code,State,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,Pizza
0,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,Accommodation & Food Services,Greeley,40.396290,-104.696990,Carl's Jr.,80631,CO,0,0,0,0,0,0,0,0
1,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,Accommodation & Food Services,Grand Junction,39.081350,-108.586890,Del Taco,81505,CO,0,0,0,1,0,0,0,0
2,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,Accommodation & Food Services,Grand Junction,39.091480,-108.554110,Which Wich,81506,CO,0,0,0,0,1,0,0,0
3,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,Accommodation & Food Services,Aurora,39.763690,-104.776710,Chipotle Mexican Grill,80011,CO,0,0,0,1,0,0,0,0
4,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,Accommodation & Food Services,Denver,39.740440,-104.926360,Taco Bell,80220,CO,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2016-05-31T13:33:45Z,2018-12-03T09:04:19Z,1803 Carolina Ave,Accommodation & Food Services,Bessemer,33.399574,-86.951050,Church's Chicken Of Birmingham,35020,AL,1,0,0,0,0,0,0,0
9996,2017-07-18T20:19:50Z,2018-12-02T00:08:36Z,8511 Cantilever Way Ste 105,Accommodation & Food Services,Raleigh,35.884500,-78.737541,Luxury Box,27613-6996,NC,1,1,0,1,1,0,0,0
9997,2017-09-05T23:05:50Z,2018-12-01T07:38:03Z,2601 Plover Rd,Accommodation & Food Services,Plover,44.453180,-89.543170,McDonald's,54467,WI,0,0,1,0,0,0,0,0
9998,2016-03-31T02:43:12Z,2018-12-01T07:35:22Z,170 E Main St,Accommodation & Food Services,Hendersonville,36.308262,-86.607339,Pizza Hut,37075,TN,1,0,0,0,0,0,0,1


In [ ]:
write_csv(fastfood_df)

Created file: "_.csv"
Suck-sess!


In [ ]:
pd.merge_ordered(walmart_cleaned, fastfood_cleaned, how='outer', on='Zip Code', fill_method='ffill')

,Latitude_x,Longitude_x,Location,Address_x,Car Juice,Zip Code,WhenAdded,LastUpdated,Address_y,Categories,PrimaryCategories,City,Latitude_y,Longitude_y,Restaurant Name,State
0,-79.290933,43.783690,"Walmart; #3000,","3850 Sheppard Ave E, Scarborough ON,M1T 3L4",0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-72.574444,42.174367,"Walmart SC; #5278,","591 Memorial Dr; I-90 Exit 5, Chicopee, MA,01020",0,01020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-72.553044,42.354910,"Walmart; #2683,","337 Russell St, Hadley, MA,01035",0,01035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-72.643020,42.341531,"Walmart; #2901,","180 N King St; I-91 Exit 21, Northhampton, MA,...",0,01060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-72.278953,42.238391,"Walmart SC; #2386,","352 Palmer Rd, Ware, MA,01082",0,01082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16846,-147.688971,64.857356,"Walmart SC; #2722,","537 Johansen Expy, Fairbanks, AK,99701",0,99709,2019-02-26T08:36:39Z,2019-04-26T09:34:26Z,3428 Airport Way,"Carry-out food,American Restaurants,Takeout,Ba...",Accommodation & Food Services,Fairbanks,64.83703,-147.80067,AW Restaurants,AK
16847,-147.688971,64.857356,"Walmart SC; #2722,","537 Johansen Expy, Fairbanks, AK,99701",0,99801,2017-07-24T21:23:05Z,2019-05-12T07:54:49Z,8585 Old Dairy Rd Ste 105,"Bars Clubs,Asian Restaurants,Japanese Restaura...",Accommodation & Food Services,Juneau,58.36360,-134.57510,Canton House,AK
16848,-147.688971,64.857356,"Walmart SC; #2722,","537 Johansen Expy, Fairbanks, AK,99701",0,99835,2015-11-06T09:59:23Z,2019-04-14T06:16:41Z,913 Halibut Point Rd,"Ice Cream Parlors,Carry-out food,Breakfast,Hot...",Accommodation & Food Services,Sitka,57.06010,-135.34930,McDonald's,AK
16849,-131.720364,55.376113,"Walmart; #2710,","4230 Don King Rd, Ketchikan, AK,99901",0,99901,2017-12-23T00:00:46Z,2019-05-19T21:43:52Z,2417 Tongass Ave Ste 120,"Carry-out food,Mexican Restaurants,Tex-mex res...",Accommodation & Food Services,Ketchikan,55.34840,-131.67413,Taco Time,AK


In [ ]:
### End of fastfood dataframe cleaning

In [ ]:
foodaccess_path = 'Outside_data/FoodAccessResearchAtlasData2019.xlsx'

xlsx = pd.ExcelFile(foodaccess_path)
print(xlsx.sheet_names)

foodaccess_xlsx = pd.read_excel(foodaccess_path, engine='xlrd')
foodaccess_df_raw = pd.DataFrame(foodaccess_xlsx)
# foodaccess_df_raw

['Read Me', 'Variable Lookup', 'Food Access Research Atlas']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.
  


In [ ]:
# Delete
# food_access_df = xlsx.parse('Food Access Research Atlas')
food_access_df.columns

Index(['CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010',
       'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10',
       ...
       'TractSeniors', 'TractWhite', 'TractBlack', 'TractAsian', 'TractNHOPI',
       'TractAIAN', 'TractOMultir', 'TractHispanic', 'TractHUNV', 'TractSNAP'],
      dtype='object', length=147)

In [ ]:
food_access_raw_df = xlsx.parse('Food Access Research Atlas')
food_access_df = food_access_raw_df
# food_access_df

In [ ]:
variable_lookup = xlsx.parse('Variable Lookup')
variable_lookup

,Field,LongName,Description
0,CensusTract,Census tract,Census tract number
1,State,State,State name
2,County,County,County name
3,Urban,Urban tract,Flag for urban tract
4,POP2010,"Population, tract total",Population count from 2010 census
...,...,...,...
142,TractAIAN,Tract American Indian and Alaska Native popula...,Total count of American Indian and Alaska Nati...
143,TractOMultir,"Tract Other/Multiple race population, number",Total count of Other/Multiple race population ...
144,TractHispanic,"Tract Hispanic or Latino population, number",Total count of Hispanic or Latino population i...
145,TractHUNV,"Tract housing units without a vehicle, number",Total count of housing units without a vehicle...


In [ ]:
def row_col_filtererer(df, column_regex=None, longname_regex=None):
  '''Because the the this function gets the variable names and longnames 
  from variable_lookup, and because variable_lookup summarizes the raw df,
  this function should be run before the raw df's colums are processed'''
  # Collect columns to filter out by column_regex and longname_regex
  if column_regex is not None:
    column_to_drop = variable_lookup.iloc[:,0].str.contains(column_regex)
  if longname_regex is not None:
    longname_to_drop = variable_lookup.iloc[:,1].str.contains(longname_regex)

  try:
    final_bool = column_to_drop + longname_to_drop
  except:
    try: final_bool = column_to_drop
    except:
      pass
    try:
      final_bool = longname_to_drop
    except:
      print("Looks like you forgot a regex (or two!)")

  return df[df.columns[~final_bool]]


row_col_filtererer(food_access_df, column_regex="LAPOP|LALOW", longname_regex='(Group)|(Vehicle|Low) (access|income).*number$')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


,CensusTract,State,County,Urban,Pop2010,OHU2010,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0,0,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,1,1,1,0,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0,0,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0,0,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,0,0,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72526,56043000200,Wyoming,Washakie County,0,3326,1317,0,0,0,0,...,593.0,3106.0,6.0,15.0,0.0,27.0,172.0,309.0,61.0,64.0
72527,56043000301,Wyoming,Washakie County,1,2665,1154,0,0,0,0,...,399.0,2377.0,5.0,23.0,0.0,40.0,220.0,446.0,88.0,41.0
72528,56043000302,Wyoming,Washakie County,1,2542,1021,0,0,0,0,...,516.0,2312.0,11.0,10.0,1.0,26.0,182.0,407.0,23.0,64.0
72529,56045951100,Wyoming,Weston County,0,3314,1322,0,0,0,0,...,499.0,3179.0,15.0,10.0,1.0,47.0,62.0,91.0,47.0,34.0


In [ ]:
zip_tract_path = 'Outside_data/ZIP_TRACT_122019.xlsx'
zip_tract_df = pd.ExcelFile(zip_tract_path)

# zip_tract_df.sheet_names - only sheet name is zip_tract_122019

zip_tract_df = zip_tract_df.parse('zip_tract_122019')
zip_tract_df

,ZIP,TRACT,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,501,36103158607,0.000000,1.000000,0.000000,1.000000
1,601,72001956700,0.623135,0.416021,0.433121,0.605512
2,601,72113071700,0.188169,0.211886,0.133758,0.188271
3,601,72001956800,0.017729,0.025840,0.089172,0.020029
4,601,72001956600,0.166052,0.341085,0.337580,0.181221
...,...,...,...,...,...,...
170184,99925,2198000200,0.000000,0.000000,1.000000,1.000000
170185,99926,2198940100,0.000000,0.000000,1.000000,1.000000
170186,99927,2198000100,0.000000,0.000000,1.000000,1.000000
170187,99928,2130000100,0.000000,0.000000,1.000000,1.000000


In [ ]:
food_access_df_dup = food_access_df

food_access_tracts = food_access_df['CensusTract']
tract_zip = zip_tract_df['TRACT']

tract_zip_reverse_index = {tract_zip: i for i, tract_zip in enumerate(tract_zip)}

zips_for_tracts = []
for tract_food_zip in food_access_tracts:
  if tract_food_zip in tract_zip_reverse_index:
    zips_for_tracts.append(
        zip_tract_df.at[tract_zip_reverse_index[tract_food_zip], 'ZIP']
    )
  if tract_food_zip not in tract_zip_reverse_index:
    zips_for_tracts.append('NA')


food_access_df = food_access_df.assign(ZIP=zips_for_tracts)
# food_access_df

In [ ]:
# food_access_df_dup = food_access_df
# food_access_df = food_access_df_dup
# Filter out variables that are dependent with other variables
#xlsx.parse('Variable Lookup')['LongName']

cols = food_access_df.columns.tolist()
cols = [cols[-1]] + cols[:-1]
food_access_df = food_access_df[cols]
food_access_df.columns
# food_access_df

Index(['ZIP', 'CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010',
       'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS',
       ...
       'TractSeniors', 'TractWhite', 'TractBlack', 'TractAsian', 'TractNHOPI',
       'TractAIAN', 'TractOMultir', 'TractHispanic', 'TractHUNV', 'TractSNAP'],
      dtype='object', length=148)

In [ ]:
food_access_df.rename(columns={'ZIP': 'Zip Code'}, inplace=True)
# food_access_df

In [ ]:
# food_access_df_dup = food_access_df
# food_access_df = food_access_df_dup
cols_and_nas = {col: food_access_df[col].isna().sum() for col in food_access_df.columns}
# print(sorted(cols_and_nas.items(), key = lambda x:x[1], reverse=True))

cols_to_drop = []
for obs in cols_and_nas.items():
  if (obs[1] > 5000):
    cols_to_drop.append(obs[0])
food_access_df.drop(cols_to_drop, axis=1, inplace=True)

rows_to_drop = food_access_df.isnull().any(axis=1)
food_access_df = food_access_df[~rows_to_drop]
food_access_df.sample(n=20, axis=0).head(50)

,Zip Code,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
11984,80210,8031003003,Colorado,Denver County,1,3138,1151,0,985.0,31.389420,...,88.0,2723.0,52.0,209.0,0.0,13.0,141.0,199.0,151.0,33.0
27683,40356,21113060102,Kentucky,Jessamine County,1,6834,2495,0,269.0,3.936201,...,673.0,6350.0,236.0,36.0,1.0,5.0,206.0,203.0,96.0,479.0
63855,77401,48201420900,Texas,Harris County,1,6231,2167,0,3.0,0.048146,...,652.0,4959.0,116.0,879.0,1.0,16.0,260.0,506.0,82.0,19.0
50725,44123,39035152102,Ohio,Cuyahoga County,1,4791,1949,0,0.0,0.000000,...,530.0,2463.0,2139.0,21.0,0.0,5.0,163.0,107.0,116.0,327.0
24770,46226,18097330805,Indiana,Marion County,1,3056,1249,0,0.0,0.000000,...,197.0,1115.0,1484.0,16.0,6.0,12.0,423.0,433.0,322.0,328.0
24550,46324,18089021700,Indiana,Lake County,1,4913,1789,0,0.0,0.000000,...,517.0,3144.0,987.0,27.0,5.0,24.0,726.0,1491.0,88.0,314.0
62467,75011,48113013720,Texas,Dallas County,1,5430,1934,0,6.0,0.110497,...,522.0,3269.0,541.0,411.0,1.0,36.0,1172.0,2406.0,28.0,135.0
22768,60543,17093890201,Illinois,Kendall County,1,4389,1531,0,0.0,0.000000,...,428.0,3646.0,213.0,51.0,4.0,13.0,462.0,1086.0,35.0,178.0
6818,90734,6037670701,California,Los Angeles County,1,6882,2489,0,78.0,1.133391,...,1506.0,4785.0,355.0,1099.0,6.0,19.0,618.0,1275.0,109.0,31.0
42842,87580,35055952100,New Mexico,Taos County,0,6383,2967,0,58.0,0.908664,...,1152.0,5065.0,11.0,25.0,2.0,90.0,1190.0,3135.0,96.0,343.0


In [ ]:
write_csv(food_access_df)

Output dataframe is "food_access_df"
Suck-sess!


In [ ]:
### End of food_access_df dataframe cleaning

In [ ]:
sahie_path = 'Outside_data/sahie_2019.csv'

sahie_csv = pd.read_csv(sahie_path, on_bad_lines='skip')
sahie_df_raw = pd.DataFrame(sahie_csv)
sahie_df_working = sahie_df_raw
sahie_df_raw

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,year,version,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,...,pctui_moe,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name,Unnamed: 25
0,2019,,1,0,40,0,0,0,0,3946002,...,0.3,88.4,0.3,11.6,0.3,88.4,0.3,Alabama ...,,NaN
1,2019,,1,0,40,0,0,0,1,1409975,...,0.6,81.1,0.6,6.8,0.2,29.0,0.4,Alabama ...,,NaN
2,2019,,1,0,40,0,0,0,2,1749910,...,0.6,82.0,0.6,8.0,0.3,36.3,0.4,Alabama ...,,NaN
3,2019,,1,0,40,0,0,0,3,942752,...,0.8,80.1,0.8,4.8,0.2,19.1,0.3,Alabama ...,,NaN
4,2019,,1,0,40,0,0,0,4,2634493,...,0.4,84.9,0.4,10.1,0.3,56.7,0.4,Alabama ...,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,2019,,4,13,50,0,0,1,3,330614,...,1.4,78.1,1.4,3.9,0.3,13.8,0.4,Arizona ...,Maricopa County,NaN
10996,2019,,4,13,50,0,0,1,4,1120054,...,0.7,80.5,0.7,11.7,0.4,48.3,0.7,Arizona ...,Maricopa County,NaN
10997,2019,,4,13,50,0,0,1,5,789440,...,0.6,81.5,0.6,7.8,0.3,34.5,0.6,Arizona ...,Maricopa County,NaN
10998,2019,,4,13,50,0,0,2,0,1860758,...,0.5,87.8,0.5,12.2,0.5,87.8,0.5,Arizona ...,Maricopa County,NaN


In [ ]:
# Delete
np.array(sahie_df_raw.statefips)

array([ 1,  1,  1, ..., 56, 56, 56])

In [ ]:
### Secondary files
sahie_metadata_path = 'Outside_data/sahie_2019_metadata.csv'
sahie_metadata_csv = pd.read_csv(sahie_metadata_path, on_bad_lines='skip')
sahie_metadata = pd.DataFrame(sahie_metadata_csv)

fips_zip_path = 'Outside_data/ZIP-COUNTY-FIPS_2018-03.csv'
fips_zip_csv = pd.read_csv(fips_zip_path)
fips_zip_df = pd.DataFrame(fips_zip_csv)

# sahie_metadata
print(fips_zip_df['STCOUNTYFP'])
print(fips_zip_df['ZIP'])

0         1001
1         1001
2         1001
3         1001
4         1001
         ...  
53957    78010
53958    78010
53959    78010
53960    78020
53961    78030
Name: STCOUNTYFP, Length: 53962, dtype: int64
0        36091
1        36758
2        36006
3        36067
4        36701
         ...  
53957      820
53958      840
53959      850
53960      830
53961      802
Name: ZIP, Length: 53962, dtype: int64


In [ ]:
# merged_state_countyfip = list(zip(sahie_df_raw['statefips'], sahie_df_raw['countyfips']))
# # df = sahie_df_raw.assign(merged_fips = merged_state_countyfip)

def merge_fips(statefip, countyfip):
  if (len(countyfip) == 1):
    countyfip = f'00{countyfip}'
  if (len(countyfip) == 2):
    countyfip = '0' + countyfip

  if (len(statefip) == 1):
    statefip = '0' + statefip

  fip = statefip + countyfip
  return fip

In [ ]:
merged_fips = np.array(map(merge_fips, sahie_df_raw['statefips'].astype('str'), sahie_df_raw['countyfips'].astype('str')))
sahie_df_working = sahie_df_working.assign(FIPS = merged_fips)
sahie_df_working['FIPS'] = sahie_df_working['FIPS'].apply(int)
print(sorted(set(sahie_df_working['FIPS'])))
print(sorted(set(fips_zip_df['STCOUNTYFP'])))

[1000, 1001, 1003, 1005, 1007, 1009, 1011, 1013, 1015, 1017, 1019, 1021, 1023, 1025, 1027, 1029, 1031, 1033, 1035, 1037, 1039, 1041, 1043, 1045, 1047, 1049, 1051, 1053, 1055, 1057, 1059, 1061, 1063, 1065, 1067, 1069, 1071, 1073, 1075, 1077, 1079, 1081, 1083, 1085, 1087, 1089, 1091, 1093, 1095, 1097, 1099, 1101, 1103, 1105, 1107, 1109, 1111, 1113, 1115, 1117, 1119, 1121, 1123, 1125, 1127, 1129, 1131, 1133, 2000, 2013, 2016, 2020, 2050, 2060, 2068, 2070, 2090, 2100, 2105, 2110, 2122, 2130, 2150, 2158, 2164, 2170, 2180, 2185, 2188, 2195, 2198, 2220, 2230, 2240, 2261, 2275, 2282, 2290, 4000, 4001, 4003, 4005, 4007, 4009, 4011, 4012, 4013, 4015, 4017, 4019, 4021, 4023, 4025, 4027, 5000, 5001, 5003, 5005, 5007, 5009, 5011, 5013, 5015, 5017, 5019, 5021, 5023, 5025, 5027, 5029, 5031, 5033, 5035, 5037, 5039, 5041, 5043, 5045, 5047, 5049, 5051, 5053, 5055, 5057, 5059, 5061, 5063, 5065, 5067, 5069, 5071, 5073, 5075, 5077, 5079, 5081, 5083, 5085, 5087, 5089, 5091, 5093, 5095, 5097, 5099, 5101, 510

In [ ]:
# Returns a dictionary with fips as keys and list of all zips for each fip as values
fips_union = set(fips_zip_df['STCOUNTYFP']).intersection(set(sahie_df_working['FIPS']))
zips_in_each_fip = {}
fips_set = fips_union
for fip in fips_set:
  zips_in_each_fip[fip] = list(fips_zip_df[fips_zip_df['STCOUNTYFP'] == fip]['ZIP'])
  
dict_order = fips_zip_df[fips_zip_df['STCOUNTYFP'].isin(fips_union)]['ZIP']
dict_order = dict_order.tolist()
zips_in_each_fip = {k: zips_in_each_fip[k] for k in dict_order}

print(zips_in_each_fip)
print(zips_in_each_fip[1001])
# dict_order.tolist()

KeyError: ignored

In [ ]:
download_file('https://query.data.world/s/e3inb7voipg5l7be5csremp2py3xwv', 'Outside_data/')

'Outside_data/e3inb7voipg5l7be5csremp2py3xwv'

In [ ]:
fips_zip_csv = pd.read_csv('/Outside_data/ZIP-COUNTY-FIPS_2018-03.csv')
pd.DataFrame(fips_zip_csv)

UnicodeDecodeError: ignored

In [ ]:
def match_fip_zip(df1: dataframe, df2=None: dataframe) -> dataframe:
  '''Appends a 'Zip Code' column to a dataframe that has only a 'FIP code' column.
  
  In the original dataframe, each observation has a unique FIP code. For each FIP code,
  this function assembles a list of possible zip codes. It then builds the 'Zip Code' column
  by selecting a random Zip code from among the possible Zip codes and assigning it to be
  the Zip code corresponding to each respective FIP code.
  
  Args:
    Takes df1 and df2 as inputs
  
  Returns:
    Returns df1 with a 'Zip Code' column'''

  if df2 is None:
    df2 = pd.read_csv('https://query.data.world/s/e3inb7voipg5l7be5csremp2py3xwv')

  fips_intersection = set(df2['STCOUNTYFP']).intersection(set(df1['FIPS']))
  # Returns a dictionary with fips as keys and list of all zips for each fip as values
  zips_in_each_fip = {}
  fips_set = fips_intersection
  for fip in fips_set:
    zips_in_each_fip[fip] = list(df2[df2['STCOUNTYFP'] == fip]['ZIP'])
  
  # Reorder zips_in_each_fip dictionary so that FIP codes in dictionary have same
  # order as in df2
  dict_order = df2[df2['STCOUNTYFP'].isin(fips_intersection)]['STCOUNTYFP']
  dict_order = dict_order.tolist()
  zips_in_each_fip = {k: zips_in_each_fip[k] for k in dict_order}

  df1 = df1[df1['FIPS'].isin(fips_intersection)]
  # Randomly select zips for each fip
  lengths_of_lists = list(Counter(df1['FIPS']).values())
  np_ones = list(map(np.ones, lengths_of_lists))

  random_zip_codes = []
  for i, array in enumerate(np_ones):
    obj = random.choices(list(zips_in_each_fip.values())[i], k=len(array))
    random_zip_codes.append(np.array(obj))
  
  np_zips_concatenated = np.concatenate(random_zip_codes).astype(int)
  df1 = df1.assign(ZIPS = np_zips_concatenated)
  df1.rename(columns={'ZIPS': 'Zip Code'}, inplace=True)
  return df1

In [ ]:
sahie_df_working = match_fip_zip(sahie_df_working, fips_zip_df)
sahie_df_working

In [ ]:
ends_with_moe = sahie_df_working.columns.str.endswith('_moe')
ends_with_moe = sahie_df_working.iloc[:,ends_with_moe]

cols_to_drop = ['statefips', 'countyfips', 'state_name', 'county_name',
       'Unnamed: 25'] # Also "NIPR", "NUI", "NIC"?
cols_to_drop = cols_to_drop + list(ends_with_moe.columns)
sahie_df_working.drop(cols_to_drop, axis=1, inplace=True)
sahie_df_working

# list(map(lambda x: sahie_df_working[sahie_df_working[x] == 0], sahie_df_working.columns))

,year,version,geocat,agecat,racecat,sexcat,iprcat,NIPR,NUI,NIC,PCTUI,PCTIC,PCTELIG,PCTLIIC,merged_fips,Zip Code
366,2019,,50,0,0,0,0,46689,4366,42323,9.4,90.6,9.4,90.6,1001,36703
367,2019,,50,0,0,0,1,13902,2410,11492,17.3,82.7,5.2,24.6,1001,36008
368,2019,,50,0,0,0,2,17665,2879,14786,16.3,83.7,6.2,31.7,1001,36066
369,2019,,50,0,0,0,3,9233,1708,7525,18.5,81.5,3.7,16.1,1001,36749
370,2019,,50,0,0,0,4,29331,3753,25578,12.8,87.2,8.0,54.8,1001,36008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320293,2019,,50,5,0,2,1,432,122,310,28.2,71.8,6.9,17.6,56045,82729
320294,2019,,50,5,0,2,2,577,154,423,26.7,73.3,8.7,24.0,56045,82729
320295,2019,,50,5,0,2,3,287,89,198,31.0,69.0,5.0,11.2,56045,82730
320296,2019,,50,5,0,2,4,978,217,761,22.2,77.8,12.3,43.2,56045,82715


In [ ]:
### End of sahie_df dataframe cleaning

In [ ]:
food_env_path = 'Outside_data/FoodEnvironmentAtlas.zip (Unzipped Files)/StateAndCountyData.csv'
food_env_csv = pd.read_csv(food_env_path)
food_env_df_raw = pd.DataFrame(food_env_csv)
food_env_df_working = food_env_df_raw
food_env_df_working
#shoddy dumthicc

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga,LACCESS_POP10,18428.439690
1,1001,AL,Autauga,LACCESS_POP15,17496.693040
2,1001,AL,Autauga,PCH_LACCESS_POP_10_15,-5.056026
3,1001,AL,Autauga,PCT_LACCESS_POP10,33.769657
4,1001,AL,Autauga,PCT_LACCESS_POP15,32.062255
...,...,...,...,...,...
852805,56,WY,Total,State_Population_2014,582548.000000
852806,56,WY,Total,State_Population_2015,585668.000000
852807,56,WY,Total,State_Population_2016,584290.000000
852808,56,WY,Total,State_Population_2017,578934.000000


In [ ]:
unique_vars = np.unique(np.array(food_env_df_working['Variable_Code']))
fip_list = []
for var in unique_vars:
  fip_list.append(list(food_env_df_working[food_env_df_working['Variable_Code'] == var]['FIPS']))

# true_zips = functools.reduce(np.intersect1d, map(list, fip_list))
true_zips = set(fip_list[0]).intersection(*map(set, fip_list[1:]))
print(true_zips)

set()


In [ ]:
def revised_df_pivoter(df_in: dataframe) -> dataframe:
  '''Filters out implausible observation with non-county-level data and implausible 
  FIP codes. In dataframe this will be applied to, unique column names are one a row
  and cycle over and over down the row, as if those columns had been pivoted from being 
  columns of the dataframe to rows of the dataframe. The order of the columns in the 
  row is relatively consistent -- one column will not come before another in one instance 
  and after that other in another instance -- but in some instances, a column name is
  missing, and so the unique column names do not neatly cycle over and over down the row.
  The main part of this function pivots out the column data that is trapped in rows and 
  puts that data into columns whose names are the column names that were trapped. This 
  function creates a dictionary, and loops over the column names when they are present,
  which are the keys of the dictionary, and appends the corresponding 'Value' value to the 
  dictionary's values, and when the column's name isn't present during looping (because
  the columns in the row are otherwise ordered), the function appends an np.NaN. This
  function returns the original dataframe with the columns that were trapped in rows,
  pivoted out to be columns.
  
  Args:
    df_in
    
  Returns:
    df_in
  '''
  # Filter out implausible or undesirable values
  df_in = df_in[df_in['FIPS'] >= 1001]
  df_in = df_in[df_in['County'] != 'Total']

  unique_fips = list(np.unique(np.array(df_in['FIPS'])))
  unique_vars = list(np.unique(np.array(df_in['Variable_Code'])))
  header_list = list(df_in['Variable_Code'])
  value_list = list(df_in['Value'])
  # Creates a slimmed down dataframe with fundamental columns to append more onto
  base_df = df_in[['FIPS', 'State', 'County']].drop_duplicates()

  # Creates a dictionary where keys are column names - values are those that
  # correspond to the column if the value exists or np.NaN if it is missing
  k = 0
  header_dict = defaultdict(list)
  extended_names = len(unique_fips)*unique_vars
  for i, col in enumerate(extended_names):
    if (extended_names[i] == header_list[k]):
      header_dict[col].append(value_list[k])
      k += 1
    else:
      header_dict[col].append(np.NaN)

  # Creates dataframe out of the dictionary created above
  dict_df = pd.DataFrame.from_dict(header_dict)
  # Concatenates fundamental dataframe and the dataframe created out of the dictionary
  df_full = pd.concat([base_df, dict_df], axis=1)

  # Final cleaning
  df_full.dropna(axis=0, subset=['FIPS'], inplace=True)
  df_full = df_full[~df_full.County.str.contains('County', regex=True)]

  return df_full

revised_df_pivoter(food_env_df_working)

,FIPS,State,County,2010_Census_Population,AGRITRSM_OPS07,AGRITRSM_OPS12,AGRITRSM_RCT07,AGRITRSM_RCT12,BERRY_ACRES07,BERRY_ACRES12,...,VEG_ACRESPTH07,VEG_ACRESPTH12,VEG_FARMS07,VEG_FARMS12,VLFOODSEC_12_14,VLFOODSEC_15_17,WICS11,WICS16,WICSPTH11,WICSPTH16
0,1001.0,AL,Autauga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,1003.0,AL,Baldwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1005.0,AL,Barbour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,1007.0,AL,Bibb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,1009.0,AL,Blount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848922,51800.0,VA,Suffolk city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848931,51810.0,VA,Virginia Beach city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848940,51820.0,VA,Waynesboro city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848949,51830.0,VA,Williamsburg city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_pivoted = revised_df_pivoter(df_in=food_env_df_working)

def col_row_na_filterer(df: dataframe, col_thresh=None: float, row_thresh=None: float) -> dataframe:
  '''Takes dataframe as input and filters out columns and rows with greater than 
  specified proportion of NAs.
  
  Args:
    df_in: dataframe

    col_thresh: float - filter out columns with more NAs than this float

    row_thresh: float - filter out rows with more NAs than this float
    
  Returns:
    df
  '''
  if row_thresh is not None:
    thresh = len(df.columns) - np.ceil(row_thresh*len(df.columns))
    df.dropna(thresh=thresh, inplace=True)

  cols_to_drop = []
  if col_thresh is not None:
    for col in df.columns:
      nas = df[col].isna()
      df_sub = df[nas]
      if ((len(list(df_sub[col].isna()))/len(list(df[col]))) > col_thresh):
        cols_to_drop.append(col)
        print("Dropped {}...".format(col))

    df.drop(cols_to_drop, axis=1, inplace=True)

  return df

col_row_na_filtered_df = col_row_na_filterer(df_pivoted, 0.999, 0.999)

Dropped 2010_Census_Population...
Dropped AGRITRSM_OPS07...
Dropped AGRITRSM_OPS12...
Dropped AGRITRSM_RCT07...
Dropped AGRITRSM_RCT12...
Dropped BERRY_ACRES07...
Dropped BERRY_ACRES12...
Dropped BERRY_ACRESPTH07...
Dropped BERRY_ACRESPTH12...
Dropped BERRY_FARMS07...
Dropped BERRY_FARMS12...
Dropped CHILDPOVRATE15...
Dropped CHIPSTAX_STORES14...
Dropped CHIPSTAX_VENDM14...
Dropped CH_FOODINSEC_14_17...
Dropped CH_VLFOODSEC_14_17...
Dropped CONVS11...
Dropped CONVS16...
Dropped CONVSPTH11...
Dropped CONVSPTH16...
Dropped CSA07...
Dropped CSA12...
Dropped DIRSALES07...
Dropped DIRSALES12...
Dropped DIRSALES_FARMS07...
Dropped DIRSALES_FARMS12...
Dropped FARM_TO_SCHOOL13...
Dropped FARM_TO_SCHOOL15...
Dropped FDPIR12...
Dropped FDPIR15...
Dropped FFR11...
Dropped FFR16...
Dropped FFRPTH11...
Dropped FFRPTH16...
Dropped FMRKT13...
Dropped FMRKT18...
Dropped FMRKTPTH13...
Dropped FMRKTPTH18...
Dropped FMRKT_ANMLPROD18...
Dropped FMRKT_BAKED18...
Dropped FMRKT_CREDIT18...
Dropped FMRKT_FRVE

In [ ]:
food_env_df = match_fip_zip(col_row_na_filtered_df, None)
food_env_df

,FIPS,State,County,LACCESS_BLACK15,LACCESS_CHILD10,LACCESS_CHILD15,LACCESS_CHILD_10_15,LACCESS_HHNV10,LACCESS_HHNV15,LACCESS_HISP15,...,PCT_LACCESS_NHASIAN15,PCT_LACCESS_NHNA15,PCT_LACCESS_NHPI15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,PCT_LACCESS_SENIORS10,PCT_LACCESS_SENIORS15,PCT_LACCESS_SNAP15,PCT_LACCESS_WHITE15,Zip Code
0,1001.0,AL,Autauga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33.769657,32.062255,NaN,NaN,NaN,NaN,36051
41,1003.0,AL,Baldwin,NaN,NaN,NaN,NaN,275.060875,224.388125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36549
82,1005.0,AL,Barbour,NaN,264.826077,264.140116,-0.259023,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36311
123,1007.0,AL,Bibb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.388206,36792
164,1009.0,AL,Blount,NaN,NaN,NaN,NaN,NaN,NaN,80.773009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848922,51800.0,VA,Suffolk city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82902
848931,51810.0,VA,Virginia Beach city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83013
848940,51820.0,VA,Waynesboro city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82939
848949,51830.0,VA,Williamsburg city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82401


In [ ]:
write_csv(food_env_df)

Created file: "_.csv"
Suck-sess!


In [ ]:
### End of food_env_df cleaning

In [ ]:
abs_path = '/content/drive/MyDrive/Free time (Aaron Presser): Poverty prediction project (02 14 2022)/'
sapie_path = 'Outside_data/est20all.xls'

sapie_path = os.path.join(abs_path, sapie_path)
# xls = pd.ExcelFile(sapie_path)
# print(xls.sheet_names)

sapie_xls = pd.read_excel(sapie_path)
sapie_df_raw = pd.DataFrame(sapie_xls)
sapie_df_raw.columns = sapie_df_raw.iloc[2,:]
sapie_df_raw = sapie_df_raw.drop([0,1,2])
sapie_df_working = sapie_df_raw.reset_index(drop=True)
sapie_df_working

2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,38371394,38309115,38433673,11.9,11.9,11.9,...,14.9,67340,67251,67429,3146325,3133736,3158914,16.8,16.7,16.9
1,01,000,AL,Alabama,714568,695249,733887,14.9,14.5,15.3,...,20.7,53958,53013,54903,66169,61541,70797,23.3,21.7,24.9
2,01,001,AL,Autauga County,6242,4930,7554,11.2,8.8,13.6,...,19.3,67565,59132,75998,.,.,.,.,.,.
3,01,003,AL,Baldwin County,20189,15535,24843,8.9,6.8,11,...,16.1,71135,66540,75730,.,.,.,.,.,.
4,01,005,AL,Barbour County,5548,4210,6886,25.5,19.3,31.7,...,47.2,38866,33510,44222,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56,037,WY,Sweetwater County,3187,2435,3939,7.6,5.8,9.4,...,10.1,70583,61835,79331,.,.,.,.,.,.
3191,56,039,WY,Teton County,1207,896,1518,5.2,3.9,6.5,...,5.4,92488,82976,102000,.,.,.,.,.,.
3192,56,041,WY,Uinta County,1697,1279,2115,8.5,6.4,10.6,...,11.7,71246,61861,80631,.,.,.,.,.,.
3193,56,043,WY,Washakie County,739,537,941,9.7,7,12.4,...,16.6,58532,50245,66819,.,.,.,.,.,.


In [ ]:
merged_fips = list(map(merge_fips, sapie_df_raw['State FIPS Code'].astype('str'), sapie_df_raw['County FIPS Code'].astype('str')))
sapie_df_working = sapie_df_working.assign(FIPS = merged_fips)
sapie_df_working['FIPS'] = sapie_df_working['FIPS'].apply(int)

sapie_columns = sapie_df_working.columns.tolist()
sapie_columns = [len(sapie_df_working.columns) - 1] + list(range(3, len(sapie_df_working.columns) - 1))
sapie_df_working = sapie_df_working.iloc[:,sapie_columns]

estimate_array = sapie_df_working.columns.str.contains('Estimate')
print(estimate_array)
new_estimate_array = np.zeros(len(estimate_array))
for i in range(len(estimate_array)):
  lim = 2
  if (i == True):
    for j in range(lim):
      np.insert(estimate_array, 0, False)
      np.delete(estimate_array, -1)

print(new_estimate_array)

[False False  True False False False False False  True False False False
 False False  True False False False False False False False False  True
 False False False False False]
[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


In [ ]:
def delete_col_and_three_after(df):
  cols_to_delete = []
  for col in df.columns:
    if col.match('Estimate'):
      cols_to_delete.extend(col.index, col.index + 1, col.index + 2)
  df.drop(cols_to_delete, axis=1, inplace=True)
  return df

In [ ]:
sapie_df_working['Name'].index

Int64Index([   3,    4,    5,    6,    7,    8,    9,   10,   11,   12,
            ...
            3188, 3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3197],
           dtype='int64', length=3195)

In [ ]:
sapie_df_working.columns = [sapie_columns[-1]] + sapie_columns[:-1]
sapie_df_working.drop(['State FIPS Code', 'County FIPS Code'], axis=1, inplace=True)

,FIPS,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,38371394,38309115,38433673,11.9,11.9,11.9,...,67340,67251,67429,3146325,3133736,3158914,16.8,16.7,16.9,0
4,01,000,AL,Alabama,714568,695249,733887,14.9,14.5,15.3,...,53958,53013,54903,66169,61541,70797,23.3,21.7,24.9,1000
5,01,001,AL,Autauga County,6242,4930,7554,11.2,8.8,13.6,...,67565,59132,75998,.,.,.,.,.,.,1001
6,01,003,AL,Baldwin County,20189,15535,24843,8.9,6.8,11,...,71135,66540,75730,.,.,.,.,.,.,1003
7,01,005,AL,Barbour County,5548,4210,6886,25.5,19.3,31.7,...,38866,33510,44222,.,.,.,.,.,.,1005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,56,037,WY,Sweetwater County,3187,2435,3939,7.6,5.8,9.4,...,70583,61835,79331,.,.,.,.,.,.,56037
3194,56,039,WY,Teton County,1207,896,1518,5.2,3.9,6.5,...,92488,82976,102000,.,.,.,.,.,.,56039
3195,56,041,WY,Uinta County,1697,1279,2115,8.5,6.4,10.6,...,71246,61861,80631,.,.,.,.,.,.,56041
3196,56,043,WY,Washakie County,739,537,941,9.7,7,12.4,...,58532,50245,66819,.,.,.,.,.,.,56043
